In [4]:
import pandas as pd
import numpy as np
import numpy.linalg as LA # operasi baris elementer
from openpyxl import load_workbook # load data excel
from sklearn.feature_extraction.text import CountVectorizer # tf-idf
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer # tf-idf
from sklearn.metrics.pairwise import cosine_similarity # cosine similarity
from nltk.corpus import stopwords # preprocessing
from nltk.stem import PorterStemmer # preprocessing bahasa inggris
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #preprocessing
import string # ya buat string
from math import*

In [5]:
datadariexcel = pd.read_excel("judul kp.xlsx")
datadariexcel

,No,Kode,Nama Mahasiswa,Nim,Judul Laporan,Preprocessing
0,1,001/KP/2000,Budi Wiyarno,96534008,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
1,2,002/KP/2000,Suryana Syamsu,96534007,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
2,3,003/KP/2000,Agis Sugiri,96534006,Proses Pembuatan Bahan Peledak di PT. DAHANA (...,proses buat bahan ledak pt dahana persero tasi...
3,4,004/KP/2000,Iip S. Iskandar,99534002,Proses Pembuatan Kertas di PT. KERTAS BLABAK M...,proses buat kertas pt kertas blabak magelang
4,5,005/KP/2001,Muhimmatul Anista,97534013,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...
5,6,005/KP/2001,Mukhlisin,97534016,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...
6,7,006/KP/2001,Slamet Riyadi,96534009,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
7,8,006/KP/2001,Novi Ariani,96534012,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
8,9,006/KP/2001,Eni Wahyuningsih,97534018,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
9,10,008/KP/2001,Addin Bakhtiar,96534014,Proses Pembuatan Kertas di PT. INDAH KIAT PULP...,proses buat kertas pt indah kiat pulp paper ta...


In [6]:
stemmer = StemmerFactory().create_stemmer()  # Object stemmer
remover = StopWordRemoverFactory().create_stop_word_remover()  # objek stopword
translator = str.maketrans('', '', string.punctuation)

In [7]:
def stemmerEN(text):
    porter = PorterStemmer()
    stop = set(stopwords.words('english'))
    text = text.lower()
    text = [i for i in text.lower().split() if i not in stop]
    text = ' '.join(text)
    preprocessed_text = text.translate(translator)
    text_stem = porter.stem(preprocessed_text)
    return text_stem

def preprocess(text):
    text = text.lower()
    text_clean = remover.remove(text) #fungsi hapus stopword 
    text_stem = stemmer.stem(text_clean)
    text_stem = stemmerEN(text_stem)
    return text_stem

In [8]:
list_preprocessing_pada_judul_penelitian = []
for hasil_preprocessing in datadariexcel['Judul Laporan']:
    list_preprocessing_pada_judul_penelitian.append(preprocess(hasil_preprocessing))

In [9]:
datadariexcel["Preprocessing"] = list_preprocessing_pada_judul_penelitian
datadariexcel

,No,Kode,Nama Mahasiswa,Nim,Judul Laporan,Preprocessing
0,1,001/KP/2000,Budi Wiyarno,96534008,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
1,2,002/KP/2000,Suryana Syamsu,96534007,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
2,3,003/KP/2000,Agis Sugiri,96534006,Proses Pembuatan Bahan Peledak di PT. DAHANA (...,proses buat bahan ledak pt dahana persero tasi...
3,4,004/KP/2000,Iip S. Iskandar,99534002,Proses Pembuatan Kertas di PT. KERTAS BLABAK M...,proses buat kertas pt kertas blabak magelang
4,5,005/KP/2001,Muhimmatul Anista,97534013,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...
5,6,005/KP/2001,Mukhlisin,97534016,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...
6,7,006/KP/2001,Slamet Riyadi,96534009,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
7,8,006/KP/2001,Novi Ariani,96534012,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
8,9,006/KP/2001,Eni Wahyuningsih,97534018,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
9,10,008/KP/2001,Addin Bakhtiar,96534014,Proses Pembuatan Kertas di PT. INDAH KIAT PULP...,proses buat kertas pt indah kiat pulp paper ta...


In [10]:
datadariexcel.to_excel("judul kp.xlsx", engine="xlsxwriter", index=False)

In [11]:
data_hasil_dari_preprocessing = pd.read_excel("judul kp.xlsx")
data_hasil_dari_preprocessing

,No,Kode,Nama Mahasiswa,Nim,Judul Laporan,Preprocessing
0,1,001/KP/2000,Budi Wiyarno,96534008,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
1,2,002/KP/2000,Suryana Syamsu,96534007,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
2,3,003/KP/2000,Agis Sugiri,96534006,Proses Pembuatan Bahan Peledak di PT. DAHANA (...,proses buat bahan ledak pt dahana persero tasi...
3,4,004/KP/2000,Iip S. Iskandar,99534002,Proses Pembuatan Kertas di PT. KERTAS BLABAK M...,proses buat kertas pt kertas blabak magelang
4,5,005/KP/2001,Muhimmatul Anista,97534013,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...
5,6,005/KP/2001,Mukhlisin,97534016,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...
6,7,006/KP/2001,Slamet Riyadi,96534009,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
7,8,006/KP/2001,Novi Ariani,96534012,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
8,9,006/KP/2001,Eni Wahyuningsih,97534018,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...
9,10,008/KP/2001,Addin Bakhtiar,96534014,Proses Pembuatan Kertas di PT. INDAH KIAT PULP...,proses buat kertas pt indah kiat pulp paper ta...


In [34]:
list_tahun_pada_tahun_penelitian = []
for index, row in data_hasil_dari_preprocessing.iterrows():
    hasil = data_hasil_dari_preprocessing.ix[index,'Tahun']=row['Kode'][-4:]
    list_tahun_pada_tahun_penelitian.append(hasil)


In [35]:
datadariexcel["Tahun"] = list_tahun_pada_tahun_penelitian
datadariexcel

,No,Kode,Nama Mahasiswa,Nim,Judul Laporan,Preprocessing,Tahun
0,1,001/KP/2000,Budi Wiyarno,96534008,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2000
1,2,002/KP/2000,Suryana Syamsu,96534007,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2000
2,3,003/KP/2000,Agis Sugiri,96534006,Proses Pembuatan Bahan Peledak di PT. DAHANA (...,proses buat bahan ledak pt dahana persero tasi...,2000
3,4,004/KP/2000,Iip S. Iskandar,99534002,Proses Pembuatan Kertas di PT. KERTAS BLABAK M...,proses buat kertas pt kertas blabak magelang,2000
4,5,005/KP/2001,Muhimmatul Anista,97534013,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...,2001
5,6,005/KP/2001,Mukhlisin,97534016,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...,2001
6,7,006/KP/2001,Slamet Riyadi,96534009,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2001
7,8,006/KP/2001,Novi Ariani,96534012,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2001
8,9,006/KP/2001,Eni Wahyuningsih,97534018,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2001
9,10,008/KP/2001,Addin Bakhtiar,96534014,Proses Pembuatan Kertas di PT. INDAH KIAT PULP...,proses buat kertas pt indah kiat pulp paper ta...,2001


In [36]:
datadariexcel.to_excel("judul kp.xlsx", engine="xlsxwriter", index=False)

In [56]:
data = pd.read_excel("judul kp.xlsx")
data

,No,Kode,Nama Mahasiswa,Nim,Judul Laporan,Preprocessing,Tahun
0,1,001/KP/2000,Budi Wiyarno,96534008,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2000
1,2,002/KP/2000,Suryana Syamsu,96534007,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2000
2,3,003/KP/2000,Agis Sugiri,96534006,Proses Pembuatan Bahan Peledak di PT. DAHANA (...,proses buat bahan ledak pt dahana persero tasi...,2000
3,4,004/KP/2000,Iip S. Iskandar,99534002,Proses Pembuatan Kertas di PT. KERTAS BLABAK M...,proses buat kertas pt kertas blabak magelang,2000
4,5,005/KP/2001,Muhimmatul Anista,97534013,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...,2001
5,6,005/KP/2001,Mukhlisin,97534016,Proses Pembuatan Semen di PT. SEMEN GRESIK (pe...,proses buat semen pt semen gresik persero tbk ...,2001
6,7,006/KP/2001,Slamet Riyadi,96534009,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2001
7,8,006/KP/2001,Novi Ariani,96534012,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2001
8,9,006/KP/2001,Eni Wahyuningsih,97534018,Laporan Kerja Praktek pada Departemen Polymeri...,lapor kerja praktek departemen polymerisasi pt...,2001
9,10,008/KP/2001,Addin Bakhtiar,96534014,Proses Pembuatan Kertas di PT. INDAH KIAT PULP...,proses buat kertas pt indah kiat pulp paper ta...,2001


In [72]:
d_train = data[['Preprocessing','Tahun']].reset_index(drop=True)
d_train

,Preprocessing,Tahun
0,lapor kerja praktek departemen polymerisasi pt...,2000
1,lapor kerja praktek departemen polymerisasi pt...,2000
2,proses buat bahan ledak pt dahana persero tasi...,2000
3,proses buat kertas pt kertas blabak magelang,2000
4,proses buat semen pt semen gresik persero tbk ...,2001
5,proses buat semen pt semen gresik persero tbk ...,2001
6,lapor kerja praktek departemen polymerisasi pt...,2001
7,lapor kerja praktek departemen polymerisasi pt...,2001
8,lapor kerja praktek departemen polymerisasi pt...,2001
9,proses buat kertas pt indah kiat pulp paper ta...,2001


In [81]:
import re
import string

frequency = {}
#document_text = open('test.txt', 'r')
#text_string = document_text.read().lower()

for word in d_train.iterrows():
    count = frequency.get(word,0)
    frequency[word] = count + 1
     
frequency_list = frequency.keys()
 
for words in frequency_list:
    print (words, frequency[words])

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [9]:
ambil_data_query = pd.read_excel("queries.xlsx")
ambil_data_query = ambil_data_query['Queries'].values
ambil_data_query

array(['hubung kerabat fenetik suku asteraceae yogyakarta',
       'intervensi keras anak usia dini yogyakarta',
       'hubung patuh terapi obat kualitas hidup pasien hipertensi puskesmas mergangsan yogyakarta',
       'ancang visualisasi informasi sistem evaluasi guru'], dtype=object)

In [51]:
import math

In [52]:
x = 2
math.sqrt(x)

1.4142135623730951

In [123]:

class Engine:
    def __init__(self):
        self.cosine_score = []
        self.train_set = []  # Documents
        self.test_set = []  # Query

    def addDocument(self, word): # fungsi untuk menambahkan dokumen dataset ke dalam list train_set
        self.train_set.append(word)

    #def setQuery(self, word):  # fungsi untuk menambahkan data query ke dalam list test_Set
        #self.test_set.append(word)
   
    def jaccard_similarity(self,query, document):
        intersection = set(query).intersection(set(document))
        union = set(query).union(set(document))
        return len(intersection)/len(union)

    
    def process_score(self):
        stopWords = stopwords.words('english') 
        vectorizer = CountVectorizer()

        trainVectorizerArray = vectorizer.fit_transform(self.train_set).toarray() 
        
        testVectorizerArray = vectorizer.fit_transform(self.train_set).toarray()
        
        jc = lambda query, document: set(query).intersection(set(document)) / set(query).union(set(document))
        
        output = []
        for i in range(0, len(testVectorizerArray)):
            output.append([])
        for vector in trainVectorizerArray:
            u = 0
            for testV in testVectorizerArray:
                cosine = jc(set(testV),set(vector))
                output[u].append((cosine))
                u = u + 1
        return output
    
engine = Engine()

dokumen = [str(x) for x in data_hasil_dari_preprocessing['Preprocessing']]

namadokumen = list()

for i, konversidokumen in enumerate(dokumen):
    engine.addDocument(konversidokumen)
    namadokumen.append("Document_{}".format(i+1))
    

# for query in ambil_data_query:
#     print(query)
#     engine.setQuery(query) #input data uji
    
titles_score = engine.process_score()
ScoreTF = (pd.DataFrame(titles_score)).T
#print(ScoreTF)
ScoreTF.columns = dokumen
ScoreTF["Documents"] = namadokumen
ScoreTF["Judul Penelitian"] = data_hasil_dari_preprocessing["Judul Penelitian"].values
ScoreTF

TypeError: unsupported operand type(s) for /: 'set' and 'set'

In [121]:
df_listed = list()
for i in dokumen:
    labels = list()
    for j in ScoreTF[i]:
        if j>0.000:
            labels.append(1)
        else:
            labels.append(0)
    datadf = pd.DataFrame(ScoreTF[i])
    datadf['Documents'] = ScoreTF['Documents']
    datadf['Labels'] = labels
    #datadf['Pembimbing'] = ScoreDf['Pembimbing'].values
    datadf['Judul Penelitian'] = ScoreTF['Judul Penelitian'].values
    df_listed.append(datadf.sort_values(by=[i], ascending=False))
df_listed

TypeError: '>' not supported between instances of 'function' and 'float'

In [99]:
df_listed[1].head(10)

,hubung perilaku iman ihsan sabar syukur bahagia makna hidup,Documents,Labels,Judul Penelitian
0,1.0,Document_1,1,Hubungan Kekerabatan Fenetik Suku Asteraceae d...
1,1.0,Document_2,1,"Hubungan Perilaku Keimanan, Ihsan, Sabar, dan ..."
2,1.0,Document_3,1,Intervensi Kekerasan Terhadap Anak Usia Dini d...
3,1.0,Document_4,1,Edukasi Dampak dan Bahaya Rokok pada Siswa Sek...
4,1.0,Document_5,1,Uji Coba Modul Konseling Farmasis (KSF) untuk ...
5,1.0,Document_6,1,HUBUNGAN KEPATUHAN TERAPI OBAT TERHADAP KUALIT...
6,1.0,Document_7,1,Analisis Biaya Medis Langsung Pengobatan Strok...
7,1.0,Document_8,1,Sistem Navigasi Robot dalam Ruangan
8,1.0,Document_9,1,PERANCANGAN VISUALISASI INFORMASI UNTUK SISTEM...
9,1.0,Document_10,1,Perancangan Antarmuka Layanan Perpustakaan Ber...


In [100]:
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer()

tf = tf_vectorizer.fit_transform(alldata)
tf_terms = tf_vectorizer.get_feature_names()
print(tf.toarray())
print(tf_terms)
tfff = tf_vectorizer.vocabulary_

a = " ".join(tf_terms)
print(a)


NameError: name 'alldata' is not defined

In [ ]:
list_token = []
for jadikanstring in tf_term:
    word_token = word_tokenize(reviews)
    list_token.append(word_token)
    print(word_token)

In [ ]:
import math

In [ ]:
def jaccard_similarity(a,b):
    c = a.intersection(b)
    jc = float(len(c))/math.sqrt(len(a)+len(b)-len(c))
    return jc

In [ ]:
#print(list_token)
# list1 = ['makna','mkana']
# list2 = ['makan']
# words1 = set(list1)
# words2 = set(list2)
#print(sampel)
C1 = "hubung kerabat fenetik suku asteraceae yogyakarta "
C2 = "intervensi keras anak usia dini yogyakarta "
C3 = "hubung patuh terapi obat kualitas hidup pasien hipertensi puskesmas mergangsan yogyakarta "
C4 = "ancang visualisasi informasi sistem evaluasi guru "
C = [C1,C2,C3,C4]
for jaccard in mengambilsampel:
    jd = jaccard_similarity(set(jaccard), set(C))
    print(jaccard,search)

In [ ]:
import nltk
# mistake = "ligting"
 
# words = ['apple', 'bag', 'drawing', 'listing', 'linking', 'living', 'lighting', 'orange', 'walking', 'zoo']
print(C[0])
for word in " ".join(tf_terms):
    jd = nltk.jaccard_distance(set(word), set(C[1]))
    print(word, jd)


In [ ]:
from sklearn.metrics import jaccard_similarity_score

In [ ]:
y_pred = [0, 2, 1, 3]

In [ ]:
y_true = [0, 1, 2, 3]

In [ ]:
jaccard_similarity_score(y_true, y_pred)

In [ ]:
jaccard_similarity_score(y_true, y_pred, normalize=False)

In [ ]:
jaccard_similarity_score(np.array([[0, 1], [1, 1]]), np.ones((2, 2)))

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(alldata)

centorids = kmeans.cluster_centers_
labels = kmeans.labels_

colors = ['g','r','b','y']

plt.figure(figsize=(15,10))

for i in range(len(x)):
    print('coordinate %s'%i, x[i], 'labels %s'%labels[i])
    plt.plot(x[i][0], x[i][1], colors[labels[i]], markersize=10)
    list_cluster = dTable[(dTabel['Judul Penelitian'] == x[i][0])].index.tolist()
    plt.text(x[i][0], x[i][1], '%s' %(list_cluster))
    
plt.scatter(centroid[:, 0], centroids[:, 1], marker = 'x', s=50, linewidths=5, zorder=10)
plt.show()

In [ ]:
def search(query):
        kunci = preprocess([query])
        alldata = []
        alldata.append(set(kunci[0].split(" ")))
        dbmodel = d.DBModel()
        datanya = dbmodel.get_data_all("hasil_temuan","stemming")
        for index in datanya:
            get_values = index.values()
            without_id = get_values
            listStem1 = []
            for j in without_id:
                if j != None:
                    x = j.encode("ascii","ignore")
                    listStem1.append(x)
                alldata.append(listStem1)
            score = []
            for tf in range(1,len(alldata)):
                score.append(jaccard_distance(alldata[0],alldata[tf]))
            return score
def jaccard_similarity(a,b):
        c = a.intersection(b)
        jc = float(len(c))/math.sqrt(len(a)+len(b)-len(c))
        return jc

In [ ]:
from __future__ import division
import string
import math

tokenize = lambda doc: doc.lower().split(" ")
dokumen = [str(x) for x in data_hasil_dari_preprocessing['Preprocessing']]

all_documents = [document_0, document_1, document_2, document_3, document_4, document_5, document_6]

tokenized_documents = [tokenize(d) for d in dokumen] # tokenized docs
all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])



In [ ]:
ambil_data_query = pd.read_excel("queries.xlsx")
ambil_data_query = ambil_data_query['Queries'].values
ambil_data_query

In [ ]:
def jaccard_similarity(a,b):
    c = a.intersection(b)
    jc = float(len(c))/math.sqrt(len(a)+len(b)-len(c))
    return jc

In [ ]:
print(dokumen[9])
print(ambil_data_query[0])
jaccard_similarity(set(ambil_data_query[1]),set(dokumen[9]))

In [ ]:
class Engine:
    def __init__(self):
        self.cosine_score = []
        self.train_set = []  # Documents
        self.test_set = []  # Query

    def addDocument(self, word): # fungsi untuk menambahkan dokumen dataset ke dalam list train_set
        self.train_set.append(word)

    def setQuery(self, word):  # fungsi untuk menambahkan data query ke dalam list test_Set
        self.test_set.append(word)
        
    def jaccard_similarity(self,a,b):
        c = a.intersection(b)
        jc = float(len(c))/math.sqrt(len(a)+len(b)-len(c))
        return jc

    
    def process_score(self):
        stopWords = stopwords.words('english') 
        vectorizer = CountVectorizer()

        #transformer = TfidfTransformer()

        trainVectorizerArray = vectorizer.fit_transform(self.train_set).toarray() 
        # menghitung Bobot dokumen dataset dan uji dan kemudian disimpan dalam bentuk array 
        testVectorizerArray = vectorizer.transform(self.test_set).toarray()

        #cx = lambda a, b: round(np.inner(a, b) / (LA.norm(a) * LA.norm(b)), 3)
        #perhitungan jaccard
        
        #fungsi tanpa nama untuk normalisasi data dan definisi rumus Cosine Similarity 
        #print (testVectorizerArray)
        output = []
        for i in range(0, len(testVectorizerArray)):
            output.append([])

        for vector in trainVectorizerArray:
            # print vector
            u = 0
            
            for testV in testVectorizerArray:
                #perhitungan Cosine Similarity dalam bentuk vector dari dataset dengan query
                #yang di masukan yang kemudian mengembalikan nilai cosine ke dalam variable
                #cosine_score dalam bentuk list.
                # print testV
                #cosine = jaccard_similarity(set(vector), set(testV))
                #cosine = jaccard_similarity(vector,testV)
                #cosine = jaccard_similarity(testV,vector)
                cosine = jaccard_similarity(set(testV),set(vector))
                
                #                 self.cosine_score.append(cosine)
                #                 bulatin = (round(cosine),2)
                output[u].append((cosine))
                u = u + 1
        return output

In [119]:
print(ambil_data_query[0])
print(dokumen[0])
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)
vectorizer = CountVectorizer()
trainVectorizerArray = vectorizer.fit_transform(dokumen).toarray() 
testVectorizerArray = vectorizer.transform(dokumen).toarray()
for vector in trainVectorizerArray:
    u = 0
        
    for testV in testVectorizerArray:
        print(jaccard_similarity(set(dokumen[6]),set((dokumen[5]))))
        u = u + 1

hubung kerabat fenetik suku asteraceae yogyakarta
hubung kerabat fenetik suku asteraceae yogyakarta
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
